# Assignment: Подробнее о методах кластеризации

Представим, что международное круизное агентство "Carnival Cruise Line" решило себя разрекламировать с помощью баннеров и обратилось для этого к Вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с Вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры - легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях - Foursqare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив с этой страницы.

Нас будет интересовать файл `checkins.dat`. Открыв его, увидим следующую структуру:

```
id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

...
```

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки не содержащие координат - они неинформативны для нас:

```
id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...
```

С помощью `pandas` построим DataFrame и убедимся, что все 396632 строк с координатами считаны успешно.

In [1]:
import csv
import numpy as np
import pandas as pd

In [3]:
# Считываем дат-файл, удаляем строки без координат, и ложим все то построчно в список
with open('data/checkins.dat') as input_file:        
    newLines = []
    for line in input_file:
        newLine = [x.strip() for x in line.split('|')]
        if len(newLine) == 6 and newLine[3] and newLine[4]:
            newLines.append(newLine)

In [4]:
# создаем цсв-файл и записываем в него данные
with open('checkins.csv', 'w') as output_file:
    file_writer = csv.writer(output_file)
    file_writer.writerows(newLines)

In [5]:
data = pd.read_csv('checkins.csv', header=0)

In [10]:
len(data)

396634

In [11]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [12]:
data.tail()

,id,user_id,venue_id,latitude,longitude,created_at
396629,955561,626076,20073,40.850100,-73.866246,2012-04-13 09:56:48
396630,955892,674797,2297,33.748995,-84.387982,2012-04-13 10:56:03
396631,956377,845102,11195,42.765366,-71.467566,2012-04-13 12:08:45
396632,956119,1139114,29488,42.439479,-83.743830,2012-04-13 11:36:44
396633,956733,960666,60,42.331427,-83.045754,2012-04-13 21:56:19


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача - хороший повод познакомиться с алгоритмом `MeanShift`, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в [sklearn user guide](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html), а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте `MeanShift`, указав `bandwidth=0.1`, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

**Примечание**: на 396632 строках, кластеризация будет работать долго. Для получения корректного ответа достаточно и 100000 (~2 минуты на "среднем" ноутбуке). Быть очень терпеливым не возбраняется - результат от этого только улучшится.

In [13]:
import sklearn.cluster as cluster

In [14]:
X = data.values[0:100000, 3:5]

In [15]:
cluster1 = cluster.MeanShift(bandwidth=0.1)
cluster1.fit(X)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [16]:
# получаем метки кластеров и координаты их центра
labels = cluster1.labels_
cluster_centers = cluster1.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
n_clusters_

3230

Некоторые из получившихся кластеров содержат слишком мало точек - такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [18]:
# считаем количесвто элементов в каждом кластере
d = {}
for label in labels:
    if label not in d.keys():
        d[label] = 1
    else:
        d[label] += 1

In [19]:
# считаем количество кластеров в которых больше 15 элементов
count = 0
for key in d.keys():
    if d[key] > 15:
        count += 1
print(count)

592


При желании увидеть получившиеся результаты на карте, можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт `mapcustomizer.com` имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

```
38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...
```

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу "Carnival Cruise Line" адреса офисов:

```
33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)
```

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

**Примечание**: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [24]:
# создаем матрицу для координат кластеров с количеством элементов больше 15
clusters_select = np.ndarray(shape=(count,2))
print(len(clusters_select))

592


In [25]:
# заполняем матрицу clusters_select  
i = 0
j = 0
while i < len(cluster_centers):
    if d[i] > 15:
        clusters_select[j] = cluster_centers[i]
        j += 1
    i += 1

In [26]:
clusters_select

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ..., 
       [  41.61853175,  -88.44556818],
       [  38.65877915,  -76.8856871 ],
       [  39.2494686 ,  -77.1821271 ]])

In [27]:
# создаем и заполняем матрицу координат офисов компании
offices = np.ndarray(shape=(6,2))
offices[0] = np.array([33.751277, -118.188740])
offices[1] = np.array([25.867736, -80.324116])
offices[2] = np.array([51.503016, -0.075479])
offices[3] = np.array([52.378894, 4.885084])
offices[4] = np.array([39.366487, 117.036146])
offices[5] = np.array([-33.868457, 151.205134])

In [35]:
def distance(office, cluster):
    return np.sqrt((office[0] - cluster[0])**2 + (office[1] - cluster[1])**2)

In [36]:
# считаем растояния от каждого офисо до каждого ценра кластера, выбираем меньшое растояние и запоминаем индекс кластера
min_dist_index = 0
min_dist = 0
i = 0
while i < len(clusters_select):
    distances = [distance(office, clusters_select[i]) for office in offices]
    if min_dist == 0:
        min_dist = min(distances)
        min_dist_index = i
    else:
        if min_dist > min(distances):
            min_dist = min(distances)
            min_dist_index = i
    i += 1

In [39]:
print('Index of cluster with minimal distance to office:', min_dist_index)
print('Minimal distance between office and cluster:', min_dist)
print('Coordinates of nearest cluster -- baner', clusters_select[min_dist_index])

Index of cluster with minimal distance to office: 403
Minimal distance between office and cluster: 0.00783475816311
Coordinates of nearest cluster -- baner [ -33.86063043  151.20477593]


In [40]:
def write_answer(coordinates):
    with open("answer.txt", "w") as f:
        f.write(str(coordinates[0]) + ' ' + str(coordinates[1]))

In [41]:
write_answer(clusters_select[min_dist_index])